# Visualizations Notebook

The following notebook has as a purpose to show visualizations regarding the Education status at South America.

### Set up of libraries and importing functions from utils

In [2]:
import pandas as pd
import altair as alt
from pathlib import Path

In [3]:
from utils import (loads_enrolled_graduated,
                   loads_cae_db,
                   translate_area_del_conocimiento,
                   translates_degree,
                   translates_institution
                    )

### Loading data (all links are official from Kaggle or from Official Website of Chile).

CAE = *Credito con Aval del Estado* means State-backed loan

In [4]:
# Loading total enrolled
url_enrollment = "https://www.kaggle.com/datasets/gustavoreyesc/matriculados-en-educacin-superior-chile-2007-2025/data"
enrolled_path = Path("../data/raw/matricula.csv") #need to have the file downloaded from the url above
raw = loads_enrolled_graduated(enrolled_path)

# Loading total graduated
url_enrollment = "https://www.kaggle.com/datasets/gustavoreyesc/titulados-en-educacin-superior-chile-2007-2024"
graduated_path = Path("../data/raw/titulados.csv") #need to have the file downloaded from the url above
titulados = loads_enrolled_graduated(graduated_path)

# Loading Loans with Backup from State
url_cae = "https://portal.ingresa.cl/pages/estadisticas/CP_2024_HISTORICA_FINAL_ID.zip"
cae_path = Path("../data/raw/cae_history.txt") #need to have the file downloaded from the url above
cae = loads_cae_db(cae_path)

# Preliminar report from National Economic Prosecutor's Office 
url = "https://www.fne.gob.cl/wp-content/uploads/2025/10/00_-Resumen-Ejecutivo_2025_10_30_AM-vf.pdf"

In [5]:
raw_dfs = [raw, titulados]

In [6]:
# year column to number
for df in raw_dfs:
    df["AÑO"] = df["AÑO"].astype(str).str[-4:]
    df["AÑO"] = df["AÑO"].astype(int)
    try:
        df["TOTAL MATRÍCULA"] = df["TOTAL MATRÍCULA"] / 1000
    except KeyError:
        pass
    try:
        df["TOTAL TITULACIONES"] = df["TOTAL TITULACIONES"] / 1000
    except KeyError:
        pass

    # columns in lowercase
    cols = [col.lower() for col in df.columns]
    df.columns = cols

# **First Chart: Evolution total students of undergrads, and grad students 2007-2025**

This line chart shows the total number of enrolled students for undergrads, grads and diploma students
in Chile for the period 2007-2025

In [7]:
# For the following chart I used ChatGPT to assist me in debugging and assisting
# me in how to develop some features of the cart

var_1 = "total matrícula"

# Getting the data
cols_1 = ["año"]
df_1 = raw[raw["nivel global"] == "Pregrado"]
df_1 = df_1.groupby(cols_1)[var_1].sum().reset_index()
df_1 = df_1.query("año < 2017")

df_1.rename(columns = {"nivel global" : "Degree Type"}, inplace=True)
#values = list(range(min(df_1["año"]), max(df_1["año"]) +1 , 2))

# Chart number of enrolled during type
chart_1 = alt.Chart(df_1).mark_bar(
    color = "#97aee0",
    width = 11,
    ).encode(
    x=alt.X("año:O", axis= alt.Axis(title="Year",
                                    #values=values,
                                    labelAngle = 0,
                                    grid = False,
                                    labelFontSize = 14,
                                    titleFontSize = 16),
                    scale=alt.Scale(2007,2025)),
    y=alt.Y(f"{var_1}:Q",
            axis= alt.Axis(title="Total enrolled (thousands)",
                           values = list(range(0, 1300, 200)),
                           grid = False,
                           labelFontSize = 14,
                            titleFontSize = 16)
                           )
            ).properties(
            title = "Total Undergrads enrolled for 2007-2025",
            height = 400,
            width = 500
    ).configure_axis(
            grid = False,
            labelFontSize = 12,
            titleFontSize = 14
        ).configure_title(
            fontSize = 16
        ).configure_view(strokeWidth=0
        ).configure(background="transparent"
        ).configure_legend(
        labelFontSize = 14
        ).configure_title(
        fontSize=20)

chart_1.save("../static/enrolled.svg")
chart_1.save("../png_charts/enrolled.png")
chart_1

alt.Chart(...)

# **Second Chart: Who absorved the increment of enrolled students?**

In [8]:
c1 = "año"
c2 = "clasificación institución nivel 1"
cols_2 = [c1, c2]

# axis values
values = list(range(2007, 2026, 2))

df_2 = raw.query("`nivel global` == 'Pregrado'")
df_2 = df_2.groupby(cols_2)["total matrícula"].sum().reset_index()
df_2[c2] = df_2.apply(translates_institution, axis = 1)

i_types = ["Professional Institutes", "Technical Colleges", "Universities"]
colors = ["#5ca49f", "#9BC1bc", "#e6ebe0"]


chart_2 = alt.Chart(df_2).mark_bar(size = 14).encode(
    # CHANGE :O (Ordinal) to :Q (Quantitative)
    x = alt.X(f"{c1}:Q",
              title = "Year",
              scale=alt.Scale(domain=[2007, 2025]),
              axis= alt.Axis(
                  values=values,
                  format = '.0f')),
    y = alt.Y("total matrícula",
              title = "Total Enrolled (thousands)").stack("normalize"),
    color = alt.Color(c2,
                      legend = alt.Legend(title="Institution Type"),
                      scale = alt.Scale(
                          domain = i_types,
                          range = colors))
).properties(
    title = "Total enrolled undergrads by Tertiary establishment type 2007-2025"
).configure_axis(
    grid = False
)
chart_2

alt.Chart(...)

# **Third Chart: Evolution Total male and female students of overall enrolled**

The following chart shows the growth of male and females students during 2007-2025


In [9]:
# For the following chart I used ChatGPT to assist me in debugging and assisting
# me in how to develop some features of the cart

cols_3 = ["año"]
df_3 = raw.groupby(cols_3)[["total matrícula hombres", "total matrícula mujeres"]].sum().reset_index()



df_3 = df_3.melt(id_vars="año",
                 value_vars=["total matrícula hombres", "total matrícula mujeres"],
                 var_name="enrolled type",
                 value_name="total matrícula")

men = "total matrícula hombres"
women = "total matrícula mujeres"
values = list(range(2007, 2026, 2))

df_3["enrolled type"] = df_3["enrolled type"].apply(lambda x: "Men" if x == men else "Women")
df_3["total matrícula"] = df_3["total matrícula"] / 1000

chart_3 = alt.Chart(df_3).mark_line().encode(
    x = alt.X("año:N", axis=alt.Axis(title="Year",
                                     values=values,
                                     labelAngle = 0,
                                     grid = False,
                                     labelFontSize = 14,
                                     titleFontSize = 16)),
    y = alt.Y("sum(total matrícula)",
              axis=alt.Axis(title="Total enrolled (thousands)",
                            grid = False,
                            labelFontSize = 14,
                            titleFontSize = 16)),
    color = alt.Color("enrolled type",
                      legend=alt.Legend(title="Gender", orient = "bottom-right"),
                      scale = alt.Scale(
                          domain = ["Women", "Men"],
                          range = ["#88498f", "#779fa1"]
                      ))).properties(
        title = "Evolution of enrolled Men and Women in universities' Degrees 2007-2025",
        height = 400,
        width = 500
    ).configure_axis(
        grid = False,
        labelFontSize = 12,
        titleFontSize = 14
        ).configure_title(
            fontSize = 16
    ).configure_view(strokeWidth=0
    ).configure(background="transparent"
    ).configure_legend(
        labelFontSize = 14
    ).configure_title(
        fontSize=20)

chart_3.save("../static/men_women.svg")
chart_3.save("../png_charts/men_women.png")
chart_3


alt.Chart(...)

# **Fourth Chart: Differences of study areas between men and women**

The following chart shows the ranking of top enrolled number of students of
knowkledge areas, comparing men and women for the 2007-2025 period

In [10]:
# For the following chart I used ChatGPT to assist me in debugging and assisting
# me in how to develop some features of the cart

# For Women

cols_4 = ["año", "área del conocimiento"]
col_add = "total matrícula mujeres"
df_4 = raw.groupby(cols_4)[col_add].sum().reset_index()

df_4["área del conocimiento"] = df_4.apply(translate_area_del_conocimiento, axis=1)

df_4_final = pd.DataFrame({"año": [], "área del conocimiento": [],
                           "total matrícula mujeres": [], "rank": []})

for year in df_4["año"].unique():
    yr_df = df_4[df_4["año"] == year].copy()
    yr_df["rank"] = yr_df["total matrícula mujeres"].rank(ascending=False)
    df_4_final = pd.concat([df_4_final, yr_df])

chart_4 = alt.Chart(df_4_final).mark_line().encode(
    x = alt.X("año:O").title("Year"),
    y = alt.Y("rank").title("Rank"),
    color = alt.Color("área del conocimiento",
                      legend = alt.Legend(title="Study Field"))).properties(
                          title = "Study field preference by WOMEN"
                      )

In [11]:
# For the following chart I used ChatGPT to assist me in debugging and assisting
# me in how to develop some features of the cart

# For Men

cols_44 = ["año", "área del conocimiento"]
col_add_44 = "total matrícula hombres"
df_44 = raw.groupby(cols_44)[col_add_44].sum().reset_index()

df_44["área del conocimiento"] = df_44.apply(translate_area_del_conocimiento, axis=1)

df_44_final = pd.DataFrame({"año": [], "área del conocimiento": [],
                           "total matrícula hombres": [], "rank": []})

for year in df_44["año"].unique():
    yr_df = df_44[df_44["año"] == year].copy()
    yr_df["rank"] = yr_df["total matrícula hombres"].rank(ascending=False)
    df_44_final = pd.concat([df_44_final, yr_df])


chart_44 = alt.Chart(df_44_final).mark_line().encode(
    x = alt.X("año:O"),
    y = "rank",
    color = alt.Color("área del conocimiento",
                      legend = alt.Legend(title="Study Field"))).properties(
                          title = "Study field preference by MEN")
chart_44

final = chart_4 | chart_44

final.save("../png_charts/ranks.png")
final

alt.HConcatChart(...)

# **Fifth Chart: Where in Chile are they studying at 2025**

The following Chart shows the number of univerisity student in each region of
Chile ordered from north to South at 2025

In [12]:
# For the following chart I used ChatGPT to assist me in debugging and assisting
# me in how to develop some features of the cart

cols_5 = ["año", "región"]
value_5 = "total matrícula"
df_5 = raw.groupby(cols_5)[value_5].sum().reset_index()
df_5 = df_5[df_5["año"] == 2025]
order_regions = ['Arica y Parinacota',
                 'Tarapacá',
                 'Antofagasta', 
                 'Atacama',
                 'Coquimbo', 
                 'Valparaíso',
                 'Metropolitana',
                 "Lib. Gral. B. O'Higgins",
                 'Maule',   
                 'Ñuble',
                 'Biobío',
                 'La Araucanía',
                 'Los Ríos',
                 'Los Lagos',
                 'Aysén',
                 'Magallanes']

chart_5 = alt.Chart(df_5).mark_bar().encode(
    x = alt.X("total matrícula").title("Total enrolled"),
    y = alt.Y("región", sort= order_regions).title("Regions of Chile (South → North)")
).properties(
    title="Total enrolled university students by region for 2025",
    width = 400,
    height = 400
)
chart_5.save("../png_charts/regions.png")
chart_5

alt.Chart(...)

# **Seventh Chart: Share of Men and Women per degree type in 2025**

The following Chart shows what's the distribution of men and women per degree type in 2025

In [13]:
# For the following chart I used ChatGPT to assist me in debugging and assisting
# me in how to develop some features of the cart

cols_7 = ["año","carrera clasificación nivel 2"]
df_7 = raw[raw["año"] == 2025]
df_7 = df_7.groupby(cols_7)[["total matrícula hombres", "total matrícula mujeres"]].sum().reset_index()

df_7

df_7 = df_7.melt(id_vars= cols_7,
                 value_vars=["total matrícula hombres", "total matrícula mujeres"],
                 var_name="enrolled type",
                 value_name="total matrícula")

men = "total matrícula hombres"
women = "total matrícula mujeres"

df_7["enrolled type"] = df_7["enrolled type"].apply(lambda x: "Men" if x == men else "Women")
print(df_7["carrera clasificación nivel 2"].unique())
df_7["carrera clasificación nivel 2"] = df_7.apply(translates_degree, axis=1)

rule_50 = pd.DataFrame({"y": [0.50]})
line_50 = alt.Chart(rule_50).mark_rule(color="black").encode(
    y = "y:Q").properties(
        title="50%"
    )

order = [
    "Technical (Undergrad)",
    "Professionals (Undergrad)",
    "Diplomas (Grad)",
    "Masters (Grad)",
    "PhD (Grad)"
]
print(df_7["carrera clasificación nivel 2"].unique())
chart_7 = alt.Chart(df_7).mark_bar().encode(
    x = alt.X("carrera clasificación nivel 2:N",
              sort=order,  # <–– this enforces your custom order
              axis=alt.Axis(title="Degree Type")),
    y = alt.Y("sum(total matrícula)",
              stack="normalize",
              axis=alt.Axis(title="Total enrolled")),
    color = alt.Color("enrolled type",
                      legend=alt.Legend(title="Gender"))
).properties(
    title = "Share of Men and Women per degree in 2025",
    width = 400,
    height = 400
)

chart_77 = chart_7 + line_50
chart_77.save("../png_charts/gender_degree.png")
chart_77

['Carreras Profesionales' 'Carreras Técnicas' 'Doctorado' 'Magister'
 'Postítulo']
['Professionals (Undergrad)' 'Technical (Undergrad)' 'PhD (Grad)'
 'Masters (Grad)' 'Non-Degree (Grad)']


alt.LayerChart(...)

# **Eightth Chart: Evolution of spending  of % GDP per education type**

The following Chart shows how much Chile has been spending in primary, secondary,
and tertiary education as part of its GDP for period 2007-2021

In [14]:
# For the following chart I used ChatGPT to assist me in debugging and assisting
# me in how to develop some features of the cart

# loading data
# url = https://ourworldindata.org/grapher/education-spending?country=%7ECHL&overlay=download-data&spending_type=gdp_share&level=level_side_by_side
spend = pd.read_csv("../data/raw/chile_spending.csv")

new_names = {
 'Year': 'year',
 'Government spending on tertiary education as share of GDP': 'Tertiary',
 'Government spending on upper secondary education as share of GDP': 'secondary_1',
 'Government spending on lower secondary education as share of GDP': 'secondary_2',
 'Government spending on primary education as share of GDP': 'Primary',
 'Government spending on pre-primary education as share of GDP': 'Pre-Primary'}
spend.rename(columns=new_names, inplace=True)
spend["Secondary"] = spend["secondary_1"] + spend["secondary_2"]
spend.drop(columns=["Entity", "Code", "secondary_1", "secondary_2", "Pre-Primary"], inplace=True)

In [15]:
levels = [ "Primary", "Secondary", "Tertiary"]
colors = ["#68C2FF", "#D7E8BA", "#2E3A8A"]

spend_long = spend.melt(id_vars=["year"],
                        var_name = "level",
                        value_vars = levels)
spend_long = spend_long.query("year > 2006 & year < 2017")

In [16]:
#axis_values = list(range(2007, 2017, 2))

chart_8 = alt.Chart(spend_long).mark_line().encode(
    x = alt.X("year:N", axis = alt.Axis(
        labelAngle = 0,
        labelFontSize = 14,
        titleFontSize = 16)
        ).title("Year"),
    y = alt.Y("value", axis = alt.Axis(
        grid=False,
        labelFontSize = 14,
        titleFontSize = 16
    )).title("% of GDP"),
    color = alt.Color("level",
                      scale = alt.Scale(
                          domain = levels,
                          range = colors
                      ),
                      legend = alt.Legend(
                          orient = "bottom-right"
                      )
                      ).title("Education Level")
                      ).properties(
        title = "Level spending (% of GDP) by level 2006-2021",
        width = 500,
        height = 400).configure_axis(
            grid = False,
            labelFontSize = 12,
            titleFontSize = 14
        ).configure_view(strokeWidth=0
        ).configure(background="transparent"
        ).configure_legend(
        labelFontSize = 14
        ).configure_title(
        fontSize=20)

chart_8.save("../static/spending.svg")
chart_8.save("../png_charts/spending.png")
chart_8

alt.Chart(...)

# **CAE solitudes by income quintile**

In [17]:
query = """
select año_licitacion, quintil_ingreso_grupo_familiar as quintil, count(año_licitacion) as total_count
from cae
where (tipo_beneficiario == 'NUEVO BENEFICIARIO')
GROUP BY año_licitacion, quintil_ingreso_grupo_familiar
"""

values = ["quintil 1 y 2", "quintil 3", "quintil 4", "quintil 5"]
#colors = ["#1d7874ff", "#679289ff", "#679289ff", "#cfe795ff"]
colors = ["red", "blue", "blue", "yellow"] # This colors will be modified later in Inkscape

solicitations = cae.execute(query).df()
solicitations["total_count"] = solicitations["total_count"]/1000

chart_solicitations = alt.Chart(solicitations).mark_area().encode(
    x = alt.X("año_licitacion",
              axis = alt.Axis(format = '.0f',
                              grid = False,
                              labelFontSize = 14,
                              titleFontSize = 16,
                              values = list(range(2006, 2026, 2)))
                              ).title("Year"),
    y = alt.Y("total_count",
              axis = alt.Axis(grid = False,
                              labelFontSize = 14,
                                titleFontSize = 16)
                                ).title("Total granted loans (thousands)"),
    color = alt.Color("quintil",
                      scale = alt.Scale(domain = values, range = colors),
                      legend = alt.Legend(title = "Quintile", orient="top-right"))
).properties(
    title = "Granted State-backed loans by income quintile (thousands)",
    height = 400,
    width = 500
).configure_axis(
    grid = False,
    labelFontSize = 12,
    titleFontSize = 18
).configure_view(strokeWidth=0
).configure(background="transparent"
).configure_legend(
    labelFontSize = 14
).configure_title(
    fontSize=20)

chart_solicitations.save("../static/cae_solitudes.svg")
chart_solicitations

alt.Chart(...)

# **Graudated from Undergrads**

In [18]:
yr = "año"
institutions = "clasificación institución nivel 1"
total_grad = "total titulaciones"
titulados

all_years = list(range(2008, 2025, 2))

colors = [ "#06DDCF", "#023436", "#037971",]
types = ["Professional Institutes", "Technical Colleges", "Universities"]


grad_year = titulados[titulados["nivel global"] == "Pregrado"]
grad_year = grad_year.groupby([yr, institutions])[total_grad].sum().reset_index()
grad_year[institutions] = grad_year.apply(translates_institution, axis = 1)

grad_chart = alt.Chart(grad_year).mark_bar().encode(
    x = alt.X(f"{yr}:O",
              title = "Year",
              axis = alt.Axis(
                  labelAngle = 0,
                  values = all_years,
                  labelFontSize = 14,
                titleFontSize = 16)),
    y = alt.Y(total_grad,
              title = "Graduated students (thousands)",
              axis=alt.Axis(
                    values=list(range(0, 280,40)),
                    grid=False,
                    labelFontSize = 14,
                    titleFontSize = 16
              )),
    color = alt.Color(institutions,
                      legend = alt.Legend(
                            title = "Institution Type",
                            orient = "top-left"),
                          scale = alt.Scale(
                              domain = types,
                          range = colors)),
).properties(
    title = "Where did people graduated from 2007-2024",
    height = 400,
    width = 500
).configure_axis(
    grid = False,
    labelFontSize = 12,
    titleFontSize = 18
).configure_view(strokeWidth=0
).configure(background="transparent"
).configure_legend(
    labelFontSize = 14
).configure_title(
    fontSize=20
)
grad_chart.save("../static/grad_from.svg")
grad_chart

alt.Chart(...)

# **Heatmap with years and where did people study considering quintile and Institution**

In [20]:
query_2 = """
select año_licitacion as año, quintil_ingreso_grupo_familiar as quintil, tipo_ies as institution, count(año_licitacion) as total_count
from cae
where (tipo_beneficiario == 'NUEVO BENEFICIARIO')
GROUP BY año_licitacion, quintil_ingreso_grupo_familiar, institution
"""

values = ["quintil 1 y 2", "quintil 3", "quintil 4", "quintil 5"]
#colors = ["#1d7874ff", "#679289ff", "#679289ff", "#cfe795ff"]
colors = ["red", "blue", "blue", "yellow"] # This colors will be modified later in Inkscape

known = ["IP", "CFT"]


cae_quint_inst = cae.execute(query_2).df()
cae_quint_inst = cae_quint_inst[cae_quint_inst["institution"]!= "FF.AA."]
cae_quint_inst["institution"] = cae_quint_inst["institution"].apply(
    lambda val: "UNIVERSIDAD" if val not in known else val)

cae_quint_inst["comb"] = cae_quint_inst["quintil"] + " - " + cae_quint_inst["institution"]
# Create a DataFrame with all possible combinations of año and comb
years = list(range(2006, 2026, 2))
combinations = cae_quint_inst["comb"].unique()
all_combinations = pd.DataFrame([(year, comb) for year in years for comb in combinations], columns=["año", "comb"])

# Merge with the original DataFrame, filling missing values with 0
cae_quint_inst_full = pd.merge(all_combinations, cae_quint_inst, on=["año", "comb"], how="left")
cae_quint_inst_full["total_count"] = cae_quint_inst_full["total_count"].fillna(0)

# Create the chart
quint_inst = alt.Chart(cae_quint_inst_full).mark_rect().encode(
    x=alt.X("año:O",
            axis=alt.Axis(
                grid=False,
                values=list(range(2006, 2026, 2)),
                labelAngle = 0,
                labelFontSize = 14,
                titleFontSize = 16)
            ),
    y=alt.Y("comb",
            title="Quintile - Institution Type",
            axis = alt.Axis(
                labelFontSize = 14,
                titleFontSize = 16
            )),
    color=alt.Color("total_count:Q",
                    scale=alt.Scale(scheme="blues"),
                    title="Total Count",
                    legend = alt.Legend(orient = "right"))
).properties(
    title = "State-Backed loans by year, income quintile and Institution",
    height = 400,
    width = 500
).configure_axis(
    grid = False,
    labelFontSize = 16,
    titleFontSize = 18,
).configure_view(strokeWidth=0
).configure(background="white"
).configure_legend(
    labelFontSize = 14
).configure_title(
    fontSize=20
)


quint_inst.save("../static/quint_inst.svg")
quint_inst

alt.Chart(...)

# **Pie Chart: Relation of students who are studying programs with negative economic present value (2023)**

In [24]:
data = {"cat" : ["Negative", "Positive"], "vals" : [0.4, 0.6]}
vpn = pd.DataFrame(data)

pie_chart = alt.Chart(vpn).mark_arc().encode(
    theta = "vals",
    color = "cat"
)
pie_chart

alt.Chart(...)